# XFEL Wave Optics Simulation Tutorial - 4

L.Samoylova <liubov.samoylova@xfel.eu>, A.Buzmakov <buzmakov@gmail.com>

Tutorial course on FEL Wavefront Propagation Simulations, 05/10/2016, SOS2016 Workshop, Trieste.

Version 28/11/2015

Wave optics software is based on SRW core library <https://github.com/ochubar/SRW>, available through WPG interactive framework <https://github.com/samoylv/WPG>

## Propagation through a beamline with mirror 

### Import modules

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals


#Importing necessary modules:
import os
import sys
import copy
import time
import errno
import numpy as np
import pylab as plt

from IPython.display import Image

### change `wpg_path` to root folder of the WPG installation

In [ ]:
wpg_path = '/Users/lsamoylv/code/ghub/latest/WPG/'
sys.path.insert(0,wpg_path)

#Import base wavefront class
from wpg import Wavefront

#Gaussian beam generator
from wpg.generators import build_gauss_wavefront

#import srwl 
from wpg.srwlib import srwl

#import some helpers functions
from wpg.wpg_uti_exfl import calculate_theta_fwhm_cdr_s1
from wpg.wpg_uti_wf import calc_pulse_energy, averaged_intensity, calculate_fwhm, get_intensity_on_axis
from wpg.wpg_uti_wf import plot_wf, look_at_q_space

plt.ion()

### define helper functions

In [ ]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [ ]:
from IPython.display import Image
Image(filename='figures/grating_vls_bl.png')

## define a beamline with VLS  grating

In [ ]:
%%file tutorial4_beamline_vls_grating_0.py
# zero order (mirror) reflection

def get_beamline():
    import os
    import wpg
    from wpg import Beamline
    from wpg.optical_elements import Aperture, Drift, Lens, Empty, Use_PP, Mirror_plane, VLS_Grating
    from wpg.wpg_uti_oe import show_transmission

    wpg_path = os.path.abspath(os.path.dirname(wpg.__file__))
    data_path = 'data_wpg_tutorial_04'

    # S1 beamline layout
    # Geometry ###
    src_to_hom1 = 274.  # Distance source to HOM 1 [m]
    src_to_hom2 = 284.  # Distance source to HOM 2 [m]
    src_to_m3 = 300.    # Distance source to vertical focusing mirror of grating mono
    src_to_vls_gr = 301.
    src_to_pslit = 400. # distance source to slit in focus 

    theta_om = 9.e-3  # [rad]
    theta_m3 = 9.e-3  # [rad]

    om_mirror_length = 0.8  # [m]
    om_clear_ap = om_mirror_length * theta_om
    
    m3_mirror_length = 0.5
    m3_clear_ap = m3_mirror_length * theta_m3
    # define the beamline:
    bl0 = Beamline()

    # Define HOM1.
    aperture_x_to_y_ratio = 1
    hom1 = Aperture(
        shape='r', ap_or_ob='a', Dx=om_clear_ap, Dy=om_clear_ap / aperture_x_to_y_ratio)
    bl0.append(
        hom1, Use_PP(semi_analytical_treatment=0, zoom=1,sampling = 1/0.6))

    # Define mirror profile
    # Apply distortions.
    mirrors_path = os.path.join(wpg_path, '..','samples', 'data_common')
    hom1_wavefront_distortion = Mirror_plane(orient='x', 
                                             theta=theta_om, 
                                             length=om_mirror_length, 
                                             range_xy=om_clear_ap/aperture_x_to_y_ratio, 
                                             filename=os.path.join(mirrors_path, 'mirror1.dat'), 
                                             scale=1, delim=' ',bPlot=True)
    print('HOM1 WF distortion'); show_transmission(hom1_wavefront_distortion);

    zoom = 1.2

    bl0.append(hom1_wavefront_distortion,
               Use_PP(semi_analytical_treatment=0, zoom=zoom, sampling=zoom/0.8))

    # Free space propagation from hom1 to hom2
    hom1_to_hom2_drift = Drift(src_to_hom2 - src_to_hom1)
    bl0.append(hom1_to_hom2_drift, Use_PP(semi_analytical_treatment=0))

    # Define HOM2 as aperture.
    zoom = 1.0
    hom2 = Aperture('r', 'a', om_clear_ap, om_clear_ap / aperture_x_to_y_ratio)
    bl0.append(hom2, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 1))

    # drift to M3 mirror
    hom2_to_m3 = Drift(src_to_m3 - src_to_hom2)

    bl0.append(hom2_to_m3, Use_PP(semi_analytical_treatment=1))
    m3 = Aperture('r', 'a', m3_clear_ap, m3_clear_ap / aperture_x_to_y_ratio)
    bl0.append(m3, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 1))
    # Define mirror profile
    # Apply distortions.
    mirrors_path = 'data_common'
    # Define mirror profile
    mm3_wavefront_distortion = Mirror_plane(orient='y', 
                                             theta=theta_m3, 
                                             length=m3_mirror_length, 
                                             range_xy=m3_clear_ap/aperture_x_to_y_ratio, 
                                             filename=os.path.join(
                                             mirrors_path, 'mj37_2.dat'), 
                                             scale=5.,
                                             bPlot=True)
    print('M3 WF distortion'); show_transmission(mm3_wavefront_distortion);
    bl0.append(mm3_wavefront_distortion,
               Use_PP(semi_analytical_treatment=0))
    q_m3 = src_to_pslit - src_to_m3
    bl0.append(Lens(1e23,1./(1./src_to_m3 + 1./q_m3)), 
            Use_PP(semi_analytical_treatment=1))
    vls_gr_0 = Aperture('r', 'a', m3_clear_ap, m3_clear_ap / aperture_x_to_y_ratio)
    bl0.append(vls_gr_0, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 1.))
    vls_gr_to_foc = Drift(src_to_pslit - src_to_vls_gr)
    bl0.append(vls_gr_to_foc, Use_PP(semi_analytical_treatment=1))
    bl0.append(Empty(),Use_PP(zoom_v=0.1, sampling_v=0.1/0.1))
    return bl0


In [ ]:
%%file tutorial4_beamline_vls_grating_1.py
# m=1 reflection grating 

def get_beamline(ekev):
    import os
    import wpg
    from wpg import Beamline
    from wpg.optical_elements import Aperture, Drift, Lens, Empty, Use_PP, Mirror_plane, VLS_grating
    from wpg.wpg_uti_oe import show_transmission
    from wpg.srwlib import SRWLOptMirPl
    import numpy as np

    wpg_path = os.path.abspath(os.path.dirname(wpg.__file__))
    data_path = 'data_wpg_tutorial_04'

    # S1 beamline layout
    # Geometry ###
    src_to_hom1 = 268.8 #274.  # Distance source to HOM 1 [m]
    src_to_hom2 = 271.7 #284.  # Distance source to HOM 2 [m]
    
    src_to_m3a = 287.4    # high energy 1-3 keV #Distance source to vertical focusing mirror of grating mono
    src_to_m3b = 288.2    # low energy 0.27 1.2 keV #Distance source to vertical focusing mirror of grating mono
    src_to_vls_gr = 288.8 #
    src_to_m5 = 326.8   # SCS distribution mirror
    src_to_pslit = 387.9  #400. # distance source to slit in focus 
    
    src_to_m3 = src_to_m3b #low energy mono

    theta_om = 9.e-3   # [rad] check!
    theta_m3 = 20.e-3  # [rad]
    alpha_gr1 = 16.239e-3 # [rad] for 0.8 keV
    vls_gr1_par = [50., 0.00101, 0, 0] #Polynomial coefficients for VLS Grating Groove Density
    vls_gr2_par = [150., 0.00101, 0, 0] #Polynomial coefficients for VLS Grating Groove Density
    # calculate grating geometry
    d = 1.e-3/vls_gr1_par[0]; #period
    wl = 12.39e-10/ekev
    m = 1 #grating order
    dtheta = np.arcsin(m*wl/(2*d*np.sin(theta_m3))) #for constant deflection angle geo
    alpha = theta_m3-dtheta; beta = theta_m3+dtheta
    print('alpha, beta: {:.1f}, {:.1f} mrad'.format(alpha*1e3,beta*1e3))
    print('{:.4g}'.format(np.cos(alpha)-np.cos(beta)))
    print('{:.4g}'.format(m*wl/d))    
    om_mirror_length = 0.8  # [m]
    om_clear_ap = om_mirror_length * theta_om
    
    m3_mirror_length = 0.58
    m3_clear_ap = m3_mirror_length * theta_m3
    gr_length = 0.5
    gr_width = 0.2
    gr_clear_ap = gr_length * theta_m3
    # define the beamline:
    bl0 = Beamline()

    # Define HOM1.
    aperture_x_to_y_ratio = 1
    hom1 = Aperture(
        shape='r', ap_or_ob='a', Dx=om_clear_ap, Dy=om_clear_ap / aperture_x_to_y_ratio)
    bl0.append(
        hom1, Use_PP(semi_analytical_treatment=0, zoom=1,sampling = 1/0.6))

    # Define mirror profile
    # Apply distortions.
    mirrors_path = os.path.join(wpg_path, '..','samples', 'data_common')
    hom1_wavefront_distortion = Mirror_plane(orient='x', 
                                             theta=theta_om, 
                                             length=om_mirror_length, 
                                             range_xy=om_clear_ap/aperture_x_to_y_ratio, 
                                             filename=os.path.join(mirrors_path, 'mirror1.dat'), 
                                             scale=1, delim=' ',bPlot=True)
    print('HOM1 WF distortion'); show_transmission(hom1_wavefront_distortion);

    zoom = 1.2

    bl0.append(hom1_wavefront_distortion,
               Use_PP(semi_analytical_treatment=0, zoom=zoom, sampling=zoom/0.8))

    # Free space propagation from hom1 to hom2
    hom1_to_hom2_drift = Drift(src_to_hom2 - src_to_hom1)
    bl0.append(hom1_to_hom2_drift, Use_PP(semi_analytical_treatment=0))

    # Define HOM2 as aperture.
    zoom = 1.0
    hom2 = Aperture('r', 'a', om_clear_ap, om_clear_ap / aperture_x_to_y_ratio)
    bl0.append(hom2, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 1))

    # drift to M3 mirror
    hom2_to_m3 = Drift(src_to_m3 - src_to_hom2)

    bl0.append(hom2_to_m3, Use_PP(semi_analytical_treatment=1))
    m3 = Aperture('r', 'a', m3_clear_ap, m3_clear_ap / aperture_x_to_y_ratio)
    bl0.append(m3, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 1))
    # Define mirror profile
    # Apply distortions.
    mirrors_path = 'data_common'
    # Define mirror profile
    pm3_wavefront_distortion = Mirror_plane(orient='y', 
                                             theta=theta_m3, 
                                             length=m3_mirror_length, 
                                             range_xy=m3_clear_ap/aperture_x_to_y_ratio, 
                                             filename=os.path.join(
                                             mirrors_path, 'mj37_2.dat'), 
                                             scale=5.,
                                             bPlot=True)
    print('M3 WF distortion'); show_transmission(pm3_wavefront_distortion);
    bl0.append(pm3_wavefront_distortion,
               Use_PP(semi_analytical_treatment=0))
    q_m3 = src_to_pslit - src_to_m3
    bl0.append(Lens(1e23,1./(1./src_to_m3 + 1./q_m3)), 
            Use_PP(semi_analytical_treatment=1))
    m3_to_vls_gr = Drift(src_to_vls_gr - src_to_m3)
    bl0.append(m3_to_vls_gr, Use_PP(semi_analytical_treatment=1))
    vls_gr_0 = Aperture('r', 'a', m3_clear_ap, m3_clear_ap / aperture_x_to_y_ratio)
    #Grating Substrate (plane mirror, deflecting in vertical plane):
    gr_sub = SRWLOptMirPl(_size_tang=gr_length, _size_sag=gr_width, _ap_shape='r',
                    _nvx=0, _nvy=np.cos(alpha_gr1), _nvz=-np.sin(alpha_gr1), _tvx=0, _tvy=np.sin(alpha_gr1))    
    vls_gr_1 = VLS_grating(_mirSub=gr_sub, 
                           _m=1, 
                           _grDen=vls_gr1_par[0], 
                           _grDen1=vls_gr1_par[1], 
                           _grDen2=vls_gr1_par[2], 
                           _grDen3=vls_gr1_par[3])
    
    bl0.append(vls_gr_0, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 1.))
    bl0.append(vls_gr_1, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 1.))
    vls_gr_to_foc = Drift(src_to_pslit - src_to_vls_gr)
    bl0.append(vls_gr_to_foc, Use_PP(semi_analytical_treatment=1))
    bl0.append(Empty(),Use_PP(zoom_v=0.1, sampling_v=0.1/0.1))
    return bl0


In [ ]:
ekev = 0.77
wl = 12.39e-10/ekev
d = 1./50e3;m=1
theta_m3 = 20.e-3
# m*wl = d (cos(q-dq) - cos(q+2q))= 2*d*sin(q) sin(2q) => dq = arcsin(m*wl/(2*d*sin(q)))
dq = np.arcsin(m*wl/(2*d*np.sin(theta_m3)))
alpha = theta_m3-dq; beta = theta_m3+dq
print(dq, alpha,beta)
print('{:.4g}'.format(np.cos(alpha)-np.cos(beta)))
print('{:.4g}'.format(m*wl/d))

### define source 
### Gaussian beam

In [ ]:
out_prefix = 'gb_'
src_oe1 = 274. # Distance to the first OE [m]

# Central photon energy.
ekev = 0.77 # Energy [keV]

# Pulse parameters.
qnC = 0.1               # e-bunch charge, [nC]
pulse_duration = 9.e-15 # [s] <-is not used really, only ~coh time pulse duration has physical meaning 
pulseEnergy = 1.5e-3    # total pulse energy, J
coh_time = 2.2e-15      # [s]<-should be SASE coherence time, then spectrum will be the same as for SASE 
#coh_time = 50.e-15#2.2e-15      # [s]<-change to this value to get quasi-moncohromatic case

# Angular distribution
theta_fwhm = calculate_theta_fwhm_cdr_s1(ekev,qnC) # CDR2011 empirical formula
theta_fwhm = 6.e-6 #empiric value for linear SASE before saturation
print('divergence FWHM {0:.1f} urad'.format(theta_fwhm*1e6))

In [ ]:
# Gaussian beam parameters
wlambda = 12.4*1e-10/ekev        # wavelength 
w0 = wlambda/(np.pi*theta_fwhm)  # FWHM beam waist; 
w0_a = w0/(2*np.sqrt(np.log(2))) # sigma of amplitude
zR = (np.pi*w0_a**2)/wlambda     # Rayleigh length, max WF curvature

# expected beam radius at OE1 position to get the range of the wavefront 
print('Intensity waist size: {:.0f} um FWHM'.format(w0*1e6))
print('Rayleigh length: {:.1f} m'.format(zR))
print('expected FWHM at distance {:.1f} m: {:.2f} mm'.format(src_oe1,theta_fwhm*src_oe1*1e3))

fname = '{0:s}at_{1:.0f}_m'.format(out_prefix,src_oe1);
bname = fname

In [ ]:
bSaved=True
data_path = 'data_wpg_tutorial_04';mkdir_p(data_path)
sig_num = 4.
num_points = 400 #number of points
range_xy = w0_a * np.sqrt(1+(src_oe1/zR)**2)/np.sqrt(2)*sig_num;
print('range_xy at OE1: {:.1f} mm'.format(range_xy*1e3))
dx = 30.e-6; range_xy = dx*(num_points-1)
print('range_xy at OE1: {:.1f} mm'.format(range_xy*1e3))
nslices = 20; 

### create Gaussian beam

In [ ]:
srwl_wf = build_gauss_wavefront(num_points, num_points, nslices, ekev, -range_xy/2, range_xy/2,
                                -range_xy/2, range_xy/2 ,coh_time/np.sqrt(2), 
                                w0_a, w0_a, src_oe1,
                                pulseEn=pulseEnergy, pulseRange=8.)
wf = Wavefront(srwl_wf)
z0 = src_oe1
#defining name HDF5 file for storing wavefront
#store wavefront to HDF5 file 
if bSaved:     
    wf.store_hdf5(os.path.join(data_path,fname+'.h5')); 
    print('saving WF to {:s}'.format(os.path.join(data_path,fname+'.h5')))

xx=calculate_fwhm(wf);
print('FWHM at distance {:.1f} m: {:.2f} x {:.2f} mm2'.format(z0,xx[u'fwhm_x']*1e3,xx[u'fwhm_y']*1e3));

### show Gaussian wavefront in R- and Q-space

In [ ]:
plot_wf(wf)
look_at_q_space(wf)

### load FEL SASE pulse 
###  _copy FEL pulse from tutorial1_data folder_
### _change formatting from raw to code to load FEL beam_

### propagate through the beamline

In [ ]:
#loading beamline from file
import imp
import time

custom_beamline = imp.load_source('custom_beamline', 'tutorial4_beamline_vls_grating_1.py')
get_beamline = custom_beamline.get_beamline
print('building beamline...');t0 = time.time();
bl = get_beamline(ekev)
print('done in {:.2f} s'.format(time.time()-t0))
print(bl)

wf=Wavefront() ;wf.load_hdf5(os.path.join(data_path,fname+'.h5'))
srwl.ResizeElecField(wf._srwl_wf, 't', [0, 5, 1]);plot_wf(wf)
print('switching to frequency domain...');t0 = time.time();
srwl.SetRepresElecField(wf._srwl_wf, 'f');plot_wf(wf) 
print('done in {:.2f} s'.format(time.time()-t0))
print('resizing in frequency domain...');t0 = time.time();
srwl.ResizeElecField(wf._srwl_wf, 'f', [0, 0.33, 0.33/0.05]);plot_wf(wf)
print('done in {:.2f} s'.format(time.time()-t0))

print('propagating through beamline...');t0 = time.time();
bl.propagate(wf)
print('done in {:.2f} s'.format(time.time()-t0));plot_wf(wf)    

print('resizing in frequency domain...');t0 = time.time();
srwl.ResizeElecField(wf._srwl_wf, 'f', [0, 2., 1]);plot_wf(wf)
print('done in {:.2f} s'.format(time.time()-t0));plot_wf(wf)    
print('switching to time domain...');t0 = time.time();
srwl.SetRepresElecField(wf._srwl_wf, 't')
print('done in {:.2f} s'.format(time.time()-t0));plot_wf(wf)    
#srwl.ResizeElecField(wf._srwl_wf, 't', [0, 0.5, 1]);plot_wf(wf)

### show propagated wavefront

In [ ]:
fwhm=calculate_fwhm(wf);
print('FWHM propagated beam: {:.2f} x {:.2f} um2'.format(fwhm[u'fwhm_x']*1e6,fwhm[u'fwhm_y']*1e6));
plot_wf(wf,im_aspect='auto')
look_at_q_space(wf)    

### save propagated pulse 